In [0]:
dbutils.fs.ls("/mnt/ankandatalakebronzelayer")

[FileInfo(path='dbfs:/mnt/ankandatalakebronzelayer/production/', name='production/', size=0, modificationTime=1726981939000),
 FileInfo(path='dbfs:/mnt/ankandatalakebronzelayer/sales/', name='sales/', size=0, modificationTime=1726981370000),
 FileInfo(path='dbfs:/mnt/ankandatalakebronzelayer/sales.customers.parquet', name='sales.customers.parquet', size=1918, modificationTime=1726967773000)]

In [0]:
dbutils.fs.ls("/mnt/ankandatalakesilverlayer")

[]

In [0]:
dbutils.fs.ls("/mnt/ankandatalakegoldlayer")

[]

In [0]:
input_path = "/mnt/ankandatalakebronzelayer/sales/orders/orders.parquet"

In [0]:
df = spark.read.parquet(input_path)
display(df)

order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id
1,1,1,2024-09-01,2024-09-05,2024-09-03,1,1
2,2,2,2024-09-02,2024-09-06,null,1,2


In [0]:
from pyspark.sql. functions import from_utc_timestamp, date_format
from pyspark.sql. types import TimestampType
df = df.withColumn("order_date", date_format(from_utc_timestamp(df["order_date"].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))

In [0]:
display(df)

order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id
1,1,1,2024-09-01,2024-09-05,2024-09-03,1,1
2,2,2,2024-09-02,2024-09-06,null,1,2


Doing Transformation for all tables

In [0]:
table_name = []
for i in dbutils.fs.ls('/mnt/ankandatalakebronzelayer/sales/'):
    table_name.append(i.name.rstrip('/'))

In [0]:
table_name

['customers', 'order_items', 'orders', 'stores']

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

for i in table_name:
    path = '/mnt/ankandatalakebronzelayer/sales/' + i + '/' + i + '.parquet'
    df = spark.read.format('parquet').load(path)
    columns = df.columns
    for col in columns:
        if "Date" in col or "date" in col:
            df = df.withColumn(col, date_format(from_utc_timestamp(df[col].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))
    output_path = '/mnt/ankandatalakesilverlayer/sales/' + i + '/'
    df.write.format('delta').mode('overwrite').save(output_path)

In [0]:
display(df)

store_id,store_name,phone,email,street,city,state,zip_code
1,Downtown Bike Store,123-456-7890,downtown@example.com,123 Main St,Anytown,CA,12345
2,Uptown Bike Store,555-555-5555,uptown@example.com,456 Elm St,Othertown,NY,67890
